In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from pandas import Series, DataFrame
from selenium.webdriver.common.keys import Keys
import time

In [2]:
driver = webdriver.Chrome('C:/Users/taehee/Downloads/chromedriver_win32/chromedriver')
wait = WebDriverWait(driver, timeout=2)
driver.implicitly_wait(5)
# tripadvisor에 접근한다.
driver.get('https://www.tripadvisor.com/Attractions-g294197-Activities-Seoul.html')

try:
    driver.find_element_by_class_name('continue').click()
except:
    print("사이트 언어가 영어인가요?")

#원래 있던 리스트 창을 저장해둔다
original_window = driver.current_window_handle

## HTML을 개발자 도구로 뜯어보자

적절한 선택자를 찾아보자

### 즐길거리 상세 보기
attractions-attraction-overview-pois-PoiInfo__info--239IR 라는 class를 가진 div
안에
h3 

단, 그냥 h3로 찾으면 위에 있는 투어나 다른 추천도 모두 찾게 되므로 유의하자.

### 리스트 더 보기 see more
see more은
attractions-attraction-overview-main-TopPOIs__see_more--2Vsb- 라는 class를 가진 div
로 찾을 수 있다

In [3]:
def reviews(sample_size):
    '''
    단위 테스트 성공. 개발 완료.
    입력 : 표본 .
    맥락 : 리뷰를 추출할 즐길거리 페이지에 있다.
    출력 : 한 즐길거리에 대한 리뷰를 모아놓은 데이터 프레임

    각 리뷰에서 새로운 항목을 추출하고 싶다면, columns와 for문을 모두 수정해야한다.
    '''
    reviews = []
    columns = ['star_point','title','text','helpful', 'contributions','Date of experience','reviewer_address']

    n = 5
    while n < sample_size:
        try:
            foot = driver.find_element_by_class_name('location-review-review-list-parts-ReviewScrollController__container--3XtvE')
            foot.find_element_by_tag_name('button').click()    
            n += 5
        except:
            print("exception")

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    review_elements = soup.find_all(class_ = 'location-review-card-Card__ui_card--2Mri0 location-review-card-Card__card--o3LVm location-review-card-Card__section--NiAcw')

    for review_element in review_elements:
        try:
            review = [
                int(review_element.find('span',class_='ui_bubble_rating')['class'][1][-2:]) / 10,
                review_element.find('a', class_='location-review-review-list-parts-ReviewTitle__reviewTitleText--2tFRT').contents[0].string,
                review_element.find('q', class_='location-review-review-list-parts-ExpandableReview__reviewText--gOmRC').contents[0].string,
                review_element.find_all('span', class_='social-member-MemberHeaderStats__bold--3z3qh')[1].string,
                review_element.find_all('span', class_='social-member-MemberHeaderStats__bold--3z3qh')[0].string,
                review_element.find('span', class_='location-review-review-list-parts-EventDate__event_date--1epHa').contents[1],
                review_element.find('span', class_='social-member-common-MemberHometown__hometown--3kM9S').contents[1]
            ]
        except IndexError:
            #아무도 helpful이나 vote를 누르지 않은 에러, 무시하고 계속
            continue
        except AttributeError:
            #주소를 공개하지 않는 에러, 무시하고 계속
            continue
        reviews.append(review)

    return DataFrame(reviews, columns = columns)
    #DataFrame(reviews, columns = columns).head()

In [4]:
driver.switch_to.window(original_window)
driver.current_window_handle;

In [5]:
def load_reviews(link):
    '''
    단위 테스트 성공. 개발 완료.
    입력 : 어떤 즐길거리 번호 int, 이름 string 예: "Gyeongbokgung Palace"
    출력 : 어트랙션의 리뷰 모음. data.frame
    '''

    assert len(driver.window_handles) == 1

    #새 창을 열고 링크로 간다.
    driver.find_element_by_tag_name('h1')

    # 슬립...매우매우중요이거없으면무조건에러남.이거하나로3시간머리싸맸음.
    time.sleep(5)
    driver.execute_script("arguments[0].click();", link)
    
    for window_handle in driver.window_handles:
            if window_handle != original_window:
                driver.switch_to.window(window_handle)
                break
                
    assert len(driver.window_handles) == 2
                
    ##즐길거리별로샘플사이즈를어떻게선정할것인지?    
    
    result = reviews(10)

    #창을 닫고 리스트 창으로 돌아간다
    driver.close();
    driver.switch_to.window(original_window)
    driver.current_window_handle;
    
    return result
    #result.head()

In [14]:
def load_page(attraction_dict, sample_size, attraction_list_length):
    '''
    단위 테스트 성공. 개발 완료.
    입력 : 즐길거리 리스트
    출력 : 새롭게 갱신한 즐길거리 리스트
    '''

    driver.current_window_handle
    #모든 여행지의 리스트를 만든다
    
    try:
        main_page = driver.find_element_by_class_name('attractions-attraction-overview-pois-PoiGrid__wrapper--2H3Mo')
        attraction_elements = main_page.find_elements_by_class_name("attractions-attraction-overview-pois-PoiInfo__info--239IR")
        find_it = 'h3'
    except:
        main_page = driver.find_element_by_id('FILTERED_LIST')
        attraction_elements = main_page.find_elements_by_class_name('listing_info')

    attraction_names = []
    for attraction_element in attraction_elements:      
        try:
            link = attraction_element.find_element_by_tag_name('h3')
            attraction_name = link.text[link.text.find('.')+2 :]
        except:
            link = attraction_element.find_element_by_class_name('tracking_attraction_title')
            attraction_name = link.text
        
        attraction_names.append((attraction_name, link))

    for attraction_name, link in attraction_names:
        if attraction_list_length < sample_size:
            attraction_dict[attraction_name] = load_reviews(link)
            attraction_list_length += 1
            print(attraction_name)
        else:
            return attraction_dict, attraction_list_length
        
    return attraction_dict, attraction_list_length

### 다음페이지 버튼 찾기
attractions-attraction-overview-main-Pagination__container--PUXGq 리스트를 먼저 찾고

현재 페이지는 
attractions-attraction-overview-main-Pagination__selected--2updu 라는 class를 가진 div
로 찾을 수 있다

현재 페이지 + 1 해서 다음 페이지로 가는 a href를 찾는다

In [9]:
driver.find_element_by_class_name('attractions-attraction-overview-main-TopPOIs__see_more--2Vsb-').click()

In [16]:
# 즐길거리 sample_size개 리스트를 만드는 함수
def dict_n(sample_size):
    '''
    입력 : 원하는 표본의 크기
    출력 : 즐길거리 별 리뷰 딕셔너리리
    '''    

    #처음 한 번만 see more을 누른다.
    
    try:
        driver.find_element_by_class_name('attractions-attraction-overview-main-TopPOIs__see_more--2Vsb-').click()
    except NoSuchElementException:
        print('continue!')
    except:
        print("exception")
    
    keep_going = True
    attraction_list_length = 0
    attraction_dict = {}
    while keep_going:
        
        attraction_dict, attraction_list_length = load_page(attraction_dict, sample_size, attraction_list_length)        

        if attraction_list_length >= sample_size:
            keep_going = False
            break    
        
        # 다음 페이지로 넘어가기
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')            
        try:
            selected_page = int(soup.find("div", class_='attractions-attraction-overview-main-Pagination__selected--2updu').find('span').string)
            page_numbers = driver.find_element_by_class_name('attractions-attraction-overview-main-Pagination__container--PUXGq')
        except:
            selected_page = int(soup.find("div", class_='pageNumbers').find('span', class_='current').string)        
            page_numbers = driver.find_element_by_class_name('pageNumbers')

        print(selected_page)    
        page_numbers.find_element_by_link_text(str(selected_page + 1)).click()

            
    return attraction_dict

In [18]:
dict_20 = dict_n(5)

Gyeongbokgung Palace
Bukhansan National Park


In [23]:
#키를 파일 이름으로 저장
for key, an_attracrion_dict in dict_20.items():
    an_attracrion_dict.to_csv(key+'.txt', mode = 'w')